# Structured Streaming Demo

### Demo

In [1]:
import os
import pathlib
import findspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split


prj_dir = pathlib.Path().resolve().parent.parent
spark_home = os.path.join(prj_dir / 'spark-3.5.0-bin-hadoop3')
findspark.init(spark_home)

spark = SparkSession.builder.appName("StructuredNetworkWordCount").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/23 15:19:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In the terminal, type `nc -lk 5555` to run the netcat server, and then type in whatever you choose.

In [3]:
# Create DataFrame representing the stream of input lines from connection to localhost:5555
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 5555).load()

23/11/23 15:20:21 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

In [5]:
# Generate running word count
wordCounts = words.groupBy("word").count()

Some of the operations we can run on the structured stream:

| Operator               | Purpose                                                                                     |
|------------------------|------------------------------------------------------------------------------------------|
| query.name()           | get the unique identifier of the running query that persists across restarts from checkpoint data |
| query.id()             | get the unique identifier of the running query that persists across restarts from checkpoint data |
| query.runId()          | get the unique id of this run of the query, which will be generated at every start/restart        |
| query.recentProgress() | an array of the most recent progress updates for this query                                       |
| query.lastProgress()   | the most recent progress update of this streaming query                                           |
| spark.streams().active | get the list of currently active streaming queries                                                |
| query.stop()           | stop the query                                                                                    |

In [ ]:
# Start running the query that prints the running counts to the console
query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

23/11/23 15:21:29 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/g6/vgc6wxj13x95m3zxhrn480540000gn/T/temporary-b3ada9ce-7424-466a-a04e-f006f307e304. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/23 15:21:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|  ali|    1|
|salam|    1|
+-----+-----+



23/11/23 15:23:18 WARN TextSocketMicroBatchStream: Stream closed by localhost:5555
